In [1]:
import numpy as np
import pandas as pd
from importlib import reload
import amazon_parsing as ap
reload(ap)

<module 'amazon_parsing' from '/Users/jeremyalbright/Documents/GitHub/auto-product-ds/amazon_parsing.py'>

## Import Amazon product data and reviews
- Data description: https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/


In [4]:
json_parser = ap.AmazonJSONParser()
product_data = json_parser.parser('./input/meta_AMAZON_FASHION.json')
review_data = json_parser.parser('./input/AMAZON_FASHION.json')

## EDA

In [12]:
product_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186637 entries, 0 to 186636
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   title            186632 non-null  object
 1   brand            139957 non-null  object
 2   feature          123875 non-null  object
 3   rank             180222 non-null  object
 4   date             185001 non-null  object
 5   asin             186637 non-null  object
 6   imageURL         132017 non-null  object
 7   imageURLHighRes  132017 non-null  object
 8   description      15869 non-null   object
 9   price            17799 non-null   object
 10  also_view        11595 non-null   object
 11  also_buy         21642 non-null   object
 12  fit              4831 non-null    object
 13  details          885 non-null     object
 14  similar_item     317 non-null     object
 15  tech1            97 non-null      object
dtypes: object(16)
memory usage: 22.8+ MB


## EDA summary 

### JA results

**From my own EDA, it appears that:**
- X
- X

**Next steps**
- X

### LLM results

## Implement transformations and feature engineering

## Model specifications

I am trying to predict product success defined as:
- A rating above 4 stars one month after launch
- A rating above 4 starts one month after launch and high engagment (above the median number of reviews)
- A sales rank in the top 25% of a given product category
- A rating above 4.75 starts one month after launch

## Model evaluation



In [11]:
from IPython.display import Markdown, display

display(Markdown(f"## Final model selection test test \
                My optimal approach would be X.  \
                 The LLM's optimal approach would be:"))

## Final model selection test test                 My optimal approach would be X.  